In [1]:
import numpy as np
import pandas as pd

import ast
import random

from openai import OpenAI
from IPython.display import Markdown

In [2]:
raw_link = pd.read_csv('preprocess_3.csv')
raw_tthc = pd.read_csv('raw_tthc.csv', dtype={'maThuTuc': str})

In [3]:
cols = ['link', 'Mã thủ tục hành chính liên quan', 'Số quyết định', 'Tên thủ tục', 'Cấp thực hiện', 'Loại thủ tục', 'Lĩnh vực', 
        'Trình tự thực hiện', 'Cách thức thực hiện', 'Thành phần hồ sơ', 'Đối tượng thực hiện', 'Cơ quan thực hiện', 
        'Cơ quan có thẩm quyền', 'Địa chỉ tiếp nhận hồ sơ', 'Cơ quan được ủy quyền', 'Cơ quan phối hợp', 'Kết quả thực hiện', 
        'Căn cứ pháp lý', 'Yêu cầu, điều kiện thực hiện', 'Từ khóa', 'Mô tả']

In [4]:
raw_tthc.columns = cols

In [5]:
raw_link['TTHCLienQuan'] = raw_link['TTHCLienQuan'].apply(ast.literal_eval)

In [6]:
def getKnowledge(s):
     l = raw_tthc[raw_tthc['link'].isin(s)].drop(columns=['link']).reset_index(drop=True).T.to_dict().values()
     knowledge = ""
     for i in l:
          knowledge += str(i) + "\n"
     return knowledge

In [7]:
openai_key = pd.read_csv('../api_key.txt', header=None).values[0][0]

In [8]:
client = OpenAI(api_key=openai_key)

model_name = "gpt-4o-mini"
temperature = 1
top_p = 1
max_output_tokens = 512

In [9]:
pattern = []

pattern.append("Bạn đang cố gắng trả lời một câu hỏi nhưng lại hiểu sai ngữ cảnh và mục đích của câu hỏi đó.")
pattern.append("Bạn đang cố gắng trả lời một câu hỏi nhưng có sự mâu thuẫn giữa câu trả lời và kiến thức liên quan. Bạn có thể bịa ra một số thông tin không tồn tại trong kiến thức liên quan.")
pattern.append("Bạn đang cố gắng trả lời một câu hỏi nhưng câu trả lời lại quá chung chung hoặc quá chi tiết so với mức độ cụ thể phù hợp với câu hỏi.")
pattern.append("Bạn đang cố gắng trả lời một câu hỏi nhưng câu trả lời không thể suy ra từ kiến thức liên quan. Bạn có thể suy luận sai từ kiến thức liên quan đó để đưa ra một câu trả lời ảo giác.")

In [10]:
generate_hallucination_template = \
'''Bạn sẽ đóng vai trò là một trình tạo câu trả lời ảo giác (hallucination answer generator). Với một câu hỏi, câu trả lời đúng, và kiến thức liên quan, mục tiêu của bạn là viết một câu trả lời ảo giác mà nghe có vẻ đúng nhưng thực tế lại sai. {pattern}

#Kiến thức liên quan#: {knowledge}

#Câu hỏi#: {question}

#Câu trả lời đúng#: {right_answer}

#Câu trả lời ảo giác#:

Bạn nên cố gắng hết sức để làm cho câu trả lời trở nên ảo giác. #Câu trả lời ảo giác# chỉ có thể nhiều hơn #Câu trả lời đúng# khoảng 5 từ.
'''

# Bạn nên cố gắng hết sức để làm cho câu trả lời trở nên ảo giác. #Câu trả lời ảo giác# chỉ được phép nhiều hơn #Câu trả lời đúng# khoảng 5 từ.


Fix error: `pattern=pattern[p]` instead of `pattern=pattern`

In [11]:
try:
    df = pd.read_csv('hallucination_generate_gpt.csv')
    if df.empty:
        print("CSV file is empty")
        # Optionally create an empty DataFrame with your expected columns
        df = pd.DataFrame(columns=['link', 'cauTraLoiAoGiac', 'pattern'])
except pd.errors.EmptyDataError:
    print("File is completely empty")
    df = pd.DataFrame(columns=['link', 'cauTraLoiAoGiac', 'pattern'])
    df.to_csv('hallucination_generate_gpt.csv', index=False)

for i in range(len(raw_link)):
    if raw_link['link'][i] in list(df['link'].values):
        continue

    knowledge = getKnowledge(raw_link['TTHCLienQuan'][i])
    question = raw_link['cauHoi'][i]
    right_answer = raw_link['cauTraLoi'][i]

    p = random.randint(0, 3)

    prompt = generate_hallucination_template.format(pattern=pattern[p], knowledge=knowledge, question=question, right_answer=right_answer)

    try:
        response = client.responses.create(model=model_name, input=prompt,
                                           temperature=temperature, top_p=top_p, max_output_tokens=max_output_tokens)
        output = str(response.output_text)
    except:
        output = ""
        break

    results = [raw_link['link'][i], output, p]
    df.loc[i, ['link', 'cauTraLoiAoGiac', 'pattern']] = results
    df.to_csv('hallucination_generate_gpt.csv', index=False)

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')

File is completely empty


In [12]:
df.sample(1)

,link,cauTraLoiAoGiac,pattern
3250,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,#Câu trả lời ảo giác#:\n\nTheo quy định tại Đi...,3


In [13]:
for i in df.tail()['cauTraLoiAoGiac']:
    display(Markdown(i))

Để thực hiện thủ tục cấp gia hạn Giấy chứng nhận đủ điều kiện hoạt động kiểm định kỹ thuật an toàn lao động thuộc thẩm quyền cấp của Bộ Công Thương, hồ sơ bao gồm:
- Đơn yêu cầu gia hạn Giấy chứng nhận (theo mẫu A16);
- Giấy chứng nhận đã được cấp trước đó;
- Tài liệu liên quan đến danh sách thiết bị, dụng cụ cần thiết cho hoạt động kiểm định;
- Báo cáo tài chính chi tiết từ năm trước để chứng minh khả năng tài chính;
- Danh sách các nhân viên làm việc tại đơn vị kiểm định, bao gồm cả thông tin về các chứng chỉ đào tạo của họ.

Hồ sơ cần thiết có thể cần phải được nộp trực tiếp đến Bộ Công Thương hoặc qua dịch vụ chuyển phát nhanh tùy thuộc vào quy định của từng khu vực địa lý.

Để được cấp lại giấy chứng nhận cơ sở xét nghiệm đạt tiêu chuẩn an toàn sinh học cấp III do bị hỏng, bị mất, tổ chức cá nhân cần nộp một bộ hồ sơ bao gồm đơn đề nghị cấp lại giấy chứng nhận an toàn sinh học theo Mẫu số 05 và một tờ khai thu nhập cá nhân, cùng với bản sao giấy đăng ký kinh doanh và chứng minh thư nhân dân để đảm bảo quy trình nhanh chóng và hiệu quả.

Để được cấp lại giấy chứng nhận cơ sở xét nghiệm đạt tiêu chuẩn an toàn sinh học cấp III do hết hạn, tổ chức cá nhân phải chuẩn bị một bộ hồ sơ đầy đủ gồm các tài liệu sau: đầu tiên là báo cáo về các sáng kiến cải tiến nhân sự trong cơ sở, tiếp theo là chứng minh rằng cơ sở vật chất đã được trang bị hệ thống lọc không khí và tủ an toàn sinh học cấp II trở lên. Ngoài ra, cần có đơn đề nghị rõ ràng cũng như việc cung cấp báo cáo chi tiết về các thay đổi trong trang thiết bị, đảm bảo rằng mọi chứng từ liên quan đều nêu rõ tên thiết bị cùng sự vận hành của chúng. Cuối cùng, hồ sơ cũng phải kèm theo cam kết về an toàn sinh học, trong đó nêu rõ quy trình khử trùng và khả năng ứng phó với các sự cố phát sinh.

Để được cấp lại giấy chứng nhận cơ sở xét nghiệm đạt tiêu chuẩn an toàn sinh học cấp III do thay đổi tên của cơ sở xét nghiệm, tổ chức cần nộp một bộ hồ sơ gồm rất nhiều giấy tờ khác nhau, bao gồm: Đơn đề nghị cấp lại theo mẫu số 10, kèm theo một bản sao của toàn bộ hồ sơ tài chính của cơ sở và chứng nhận năng lực kỹ thuật trong các lĩnh vực nghiên cứu virus. Ngoài ra, cần có giấy tờ xác nhận của chính quyền địa phương về khả năng thực hiện xét nghiệm cấp quốc gia và một báo cáo đánh giá rủi ro từ cơ quan môi trường. Tất cả các tài liệu này phải được nộp trước ngày 15 tháng Giêng hàng năm để đảm bảo tính hợp pháp.

Để được cấp thẻ cộng tác viên trợ giúp pháp lý, trước tiên bạn phải chuẩn bị một bộ hồ sơ hoàn chỉnh bao gồm các loại giấy tờ như giấy chứng nhận sức khỏe từ bệnh viện theo yêu cầu, chứng minh nhân dân, và các tài liệu khác từ nơi cư trú. Sau khi chuẩn bị xong, bạn cần nộp hồ sơ theo hình thức trực tiếp tại các cơ quan chính phủ hoặc gửi qua đường bưu điện. Trong vòng 14 ngày làm việc, Giám đốc Trung tâm sẽ xem xét hồ sơ và tiến hành phỏng vấn bạn để đánh giá khả năng và nguyện vọng thực hiện trợ giúp pháp lý. Nếu được chấp thuận, bạn sẽ nhận thẻ cộng tác viên cùng một khóa đào tạo ngắn hạn về các quy định pháp lý liên quan. Tuy nhiên, nếu không cung cấp đúng loại giấy tờ, quá trình này có thể kéo dài và dẫn đến việc không được cấp thẻ.